### 檢索增強生成
- 使用dataset(roberthsu2003/data_for_RAG)
- 使用模型(使用Microsoft開源的intfloat/multilingual-e5-large)
- 模型大約要2.24GB

In [ ]:
from transformers import AutoTokenizer, AutoModel

#模型名稱
model_name = "intfloat/multilingual-e5-large"

#載入tokenizer和模型
tokenizer = AutoTokenizer.from_pretrained(model_name) #
tokenizer

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

XLMRobertaTokenizerFast(name_or_path='intfloat/multilingual-e5-large', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

# XLMRobertaTokenizerFast 介紹

XLMRobertaTokenizerFast 是一個優化版的多語言分詞器(tokenizer)，主要具有以下特點：

### 核心功能
- 支援多語言處理能力
- 採用 SentencePiece 演算法進行分詞
- 使用 Rust 實現的快速版本，比原始 Python 版本效能更好

### 重要參數說明
1. **vocab_size**: 250002
   - 詞彙表大小，包含所有可能的 token

2. **model_max_length**: 512 
   - 模型可處理的最大序列長度

3. **special_tokens**:
   - `<s>`: 句子開始標記
   - `</s>`: 句子結束標記
   - `<unk>`: 未知詞標記
   - `<pad>`: 填充標記
   - `<mask>`: 遮蔽標記(用於遮蔽語言模型預訓練)

### 使用優勢
1. **快速處理**:
   - Fast 版本比標準版本處理速度快很多
   - 特別適合處理大量文本數據

2. **多語言支援**:
   - 可以處理多種語言的文本
   - 不需要為不同語言使用不同的分詞器

3. **與 XLM-RoBERTa 模型完美整合**:
   - 專門為 XLM-RoBERTa 模型設計
   - 確保輸入數據格式正確



In [36]:
import torch
from pprint import pprint
#輸入文本,XLMRobertaTokenizerFast,要求的是list的型別
#中文和英文可以比較2個語意相似度
texts = ["This is a test sentence.", "這是一個測試句子."]

#將texts大內容編碼成為XLM-RoBERTa接受輸入的格式
#padding是讓第2維的元素數量相同
#return_tensors,讓input_ids的value是pytorch的tensor
tokenized = tokenizer(texts,truncation=True,padding=True, return_tensors='pt')
pprint(tokenized,compact=True) #類似dict類型的資料

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[     0,   3293,     83,     10,   3034, 149357,      5,      2],
        [     0,      6, 215617,  64770,  27683,   1344,      5,      2]])}


In [31]:
#載入模型
model = AutoModel.from_pretrained(model_name) #2.24GB
model

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, eleme

In [32]:
#使用模型生成輸出的嵌入(embeded)
model.eval() # 設定為評估模式
with torch.no_grad():
    outputs = model(**tokenized)
outputs #我們要的資料儲存於last_hidden_state

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1858, -0.0582, -0.8848,  ..., -0.2282, -0.8866,  0.8900],
         [ 0.2669, -0.5489, -0.7648,  ...,  0.0717, -0.6237,  0.7811],
         [-0.0197, -0.4468, -0.8202,  ..., -0.5010, -0.7877,  0.7135],
         ...,
         [-0.7589, -0.2531, -0.6186,  ..., -0.7020, -0.8006,  0.8468],
         [-0.1006, -0.1541, -0.6499,  ..., -0.4303, -0.9476,  0.6015],
         [ 0.1857, -0.0580, -0.8845,  ..., -0.2282, -0.8864,  0.8901]],

        [[ 1.3186, -0.1264, -0.6599,  ..., -0.5767, -0.6413,  0.7367],
         [ 1.2949, -0.2761, -0.7651,  ..., -0.7754, -0.6019,  0.5162],
         [ 1.3392, -0.4793, -0.5624,  ..., -0.5856, -0.5774,  0.4940],
         ...,
         [ 0.9948, -0.2463, -0.7340,  ..., -0.7638, -0.4298,  0.4737],
         [ 0.6910, -0.1470, -0.4917,  ..., -0.6273, -0.8047,  0.4675],
         [ 1.3187, -0.1263, -0.6597,  ..., -0.5768, -0.6413,  0.7367]]]), pooler_output=tensor([[-0.7886,  0.5136, -0.0641,  .

# `torch.no_grad()` 的用途說明

### 主要目的
`torch.no_grad()` 是一個上下文管理器 (context manager)，主要有以下用途：

1. **停用梯度計算**
   - 在推論 (inference) 階段不需要計算梯度
   - 減少記憶體使用量
   - 加快運算速度

2. **記憶體優化**
   - 不會儲存計算圖 (computational graph)
   - 不會為反向傳播 (backpropagation) 保存中間結果

### 使用時機

1. **模型評估階段**


In [ ]:
model.eval()  # 設定為評估模式
with torch.no_grad():
    predictions = model(test_data)



2. **推論階段**
   - 當我們只需要模型的輸出結果
   - 不需要進行模型訓練時

### 效能比較

使用 `torch.no_grad()` 的好處：
- 記憶體使用量可減少約一半
- 計算速度提升約20%-30%
- 特別適合處理大型模型或大量數據

### 注意事項

1. **訓練時不要使用**
   - 訓練階段需要梯度計算
   - 使用 `no_grad()` 會影響模型學習

2. **使用範圍**
   - 只在需要推論的代碼區塊使用
   - 可以和 `model.eval()` 配合使用

In [ ]:
#提取嵌入結果
embeddings = outputs.last_hidden_state #embeddings是一個tensor
print("嵌入向量的形狀:",embeddings.shape) #3維資料,2筆數據,每一筆有16個序列,每一個分詞有1024個資料 [批次大小, 序列長度, 隱藏層大小]

嵌入向量的形狀: torch.Size([2, 16, 1024])


In [33]:
#提取嵌入平均
sentence_embeddings = embeddings.mean(dim=1)
sentence_embeddings
print(f"sentence_embeddings的形狀:{sentence_embeddings.shape}")

sentence_embeddings的形狀:torch.Size([2, 1024])


# 向量平均的計算說明

這段程式碼在計算句子的最終向量表示，讓我們拆解來看：

### 程式碼解析

1. **embeddings 的結構**:
   - 是一個 3 維張量 (tensor)
   - 維度分別代表：`[批次大小, 序列長度, 隱藏層大小]`
   - 在這個例子中可能是：`[2, 16, 1024]`
     - 2：處理 2 個句子
     - 16：每個句子被分成 16 個 token
     - 1024：每個 token 的向量維度

2. **mean(dim=1) 的作用**:
   - `dim=1` 表示在第二個維度（序列長度）上取平均
   - 將每個句子的所有 token 向量平均起來
   - 結果會得到 `[2, 1024]` 的張量
     - 2：仍然是 2 個句子
     - 1024：每個句子的最終向量表示

### 實際例子
假設我們有以下結構：


In [ ]:
# 原始 embeddings 結構
[
    # 第一個句子的 16 個 token 向量
    [[0.1, 0.2, ...], [0.3, 0.4, ...], ...], 
    # 第二個句子的 16 個 token 向量
    [[0.5, 0.6, ...], [0.7, 0.8, ...], ...]
]

# 取平均後的 sentence_embeddings
[
    [0.25, 0.35, ...],  # 第一個句子的平均向量
    [0.65, 0.75, ...]   # 第二個句子的平均向量
]



這種平均操作可以將不同長度的句子轉換成相同維度的向量表示，便於後續的相似度比較或其他處理。

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
# 計算餘弦相似度
similarity = cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1].unsqueeze(0))
similarity

array([[0.87943894]], dtype=float32)

# Tensor 維度擴展 - unsqueeze 說明

### unsqueeze(0) 的功能
`unsqueeze(0)` 是 PyTorch 中用於在指定位置增加一個維度的方法。數字 `0` 表示在第一個維度（最外層）增加一個維度。

### 實際範例解析

假設原始 tensor 形狀為：


In [ ]:
sentence_embeddings[0]  # shape: [1024]



使用 unsqueeze(0) 後：


In [ ]:
sentence_embeddings[0].unsqueeze(0)  # shape: [1, 1024]



### 為什麼需要 unsqueeze?

1. **批次處理需求**
   - 很多 PyTorch 函數要求輸入必須是批次形式
   - 單個樣本需要轉換為 [1, features] 的形狀

2. **計算相似度時的維度匹配**
   - 在計算餘弦相似度時常需要這種形狀
   - 確保維度對齊以進行正確的矩陣運算

### 程式碼示例


In [ ]:
# 原始向量
vector = torch.tensor([1, 2, 3])  # shape: [3]

# 增加維度
expanded = vector.unsqueeze(0)    # shape: [1, 3]

# 用於計算相似度
similarity = torch.cosine_similarity(expanded, other_vectors)



### 常見使用場景
- 模型推論時的單樣本處理
- 向量相似度計算
- 批次處理操作